In [ ]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
df = pd.read_csv('data/wine_data.csv')
features = df.drop('Class', axis=1)
labels = df[['Class']]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=0)

Xtrain_ = torch.from_numpy(X_train.values).float()
Xtest_ = torch.from_numpy(X_test.values).float()
ytrain_ = torch.from_numpy(y_train.values).long().view(1, -1)[0]
ytest_ = torch.from_numpy(y_test.values).long().view(1, -1)[0]

input_size = 13
output_size = 3
hidden_size = 100

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
    

    def forward(self, X):
        X = torch.sigmoid(self.fc1(X))
        X = torch.sigmoid(self.fc2(X))
        X = self.fc3(X)

        print(f'Inside the model: Device id - {torch.cuda.current_device()}')

        return F.log_softmax(X, dim=-1)

In [ ]:
Xtrain_.to(device)
Xtest_.to(device)
ytrain_.to(device)
ytest_.to(device)

In [ ]:
model = Net()
model = nn.DataParallel(model)
model.to(device)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.NLLLoss()


In [ ]:
epochs = 100
for epoch in range(epochs):

    optimizer.zero_grad()
    y_pred = model(Xtrain_)
    loss = loss_fn(y_pred, ytrain_)
    loss.backward()
    optimizer.step()

    print(f'Outside the model: Device id - {torch.cuda.current_device()}| Epoch: {epoch} | Loss - {loss.item()}')

In [ ]:
with torch.no_grad():
    predict_out = model(Xtest_)
    _, predict_y = torch.max(predict_out, 1)